In [90]:
import re
import os
import requests

import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime

In [91]:
def parse_tw(bs):
    entries = []
    for row in bs.select('.main-header-info'):
        link = row.select('.posting-title a')[0]
        entries.append({
            'title': link.text,
            'link': 'https://us63.dayforcehcm.com' + link.attrs['href'],
            'ref': 'tw-' + re.search(r'Req #(\d+)', row.select('.posting-subtitle div')[0].text).group(1),
            'date': row.select('.posting-date')[0].text,
        })

    return entries

In [92]:
def merge(df, df_prev):
    for index, row in df_prev.loc[~df_prev.index.isin(df.index)].iterrows():
        if len(row.closed) == 0:
            df_prev.loc[index, ['closed']] = datetime.now().strftime('%Y-%m-%d')

    return pd.concat(
        [
            df_prev,
            df.iloc[~df.index.isin(df_prev.index)]
        ]
    ).sort_values(by='ref', ascending=False)

In [93]:



file_path = '../../data/scraps/jobs.csv'
parsers = [
    ('https://us63.dayforcehcm.com/CandidatePortal/en-US/packportal/', parse_tw)
]


df = None
for url, parser in parsers:
    response = requests.get(url)
    bs = BeautifulSoup(response.text)

    df_temp = pd.DataFrame(parser(bs)).set_index('ref')
    df_temp['closed'] = ''

    if df is None:
        df = df_temp.copy()
    else:
        df = pd.concat([df, df_temp]).drop_duplicates()


if os.path.exists(file_path):
    df_prev = pd.read_csv(file_path, index_col='ref')
    df_prev['closed'] = df_prev['closed'].fillna('')
else:
    df_prev = df.copy()


merge(df, df_prev).sort_values(by='ref', ascending=False).to_csv(file_path)




In [94]:
pd.read_csv(file_path, index_col='ref').drop(columns=['link']).fillna('')

,title,date,closed
ref,,,
tw-182,Human Resources Coordinator,"Wednesday, October 26, 2022",2022-11-04
tw-181,Iowa Activation Manager,"Monday, October 24, 2022",
tw-180,Lynx Group Events Coordinator,"Friday, October 14, 2022",
tw-178,Executive Producer of Digital Video Production,"Tuesday, October 11, 2022",
tw-177,Partnership Sales Director,"Tuesday, October 11, 2022",
tw-172,"Account Executive, Lynx Sales and Service","Monday, September 12, 2022",
tw-157,Network Engineer,"Thursday, October 6, 2022",
